<a href="https://colab.research.google.com/github/edoucet-AI/project_1_eda/blob/main/project_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import the required libraries and dependencies
import pandas as pd
import datetime as dt
from prophet import Prophet

%matplotlib inline

In [ ]:
# Read the data into a Pandas DataFrame

# Review the DataFrame
